# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import sqlite3
import sqlalchemy
import sys
import io



from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# Connect to known databases
conn_chrismoetable = sqlite3.connect('chrismoetable.db')
conn_disasterresponse = sqlite3.connect('DisasterResponse.db')
conn_insertdb = sqlite3.connect('InsertDatabaseName.db')

def list_tables(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return connection.execute(query).fetchall()

# List tables for each database
chrismoetable_tables = list_tables(conn_chrismoetable)
disasterresponse_tables = list_tables(conn_disasterresponse)
insertdatabase_tables = list_tables(conn_insertdb)

# Close the connections
conn_chrismoetable.close()
conn_disasterresponse.close()
conn_insertdb.close()

# Output the results
print(chrismoetable_tables)
print(disasterresponse_tables)
print(insertdatabase_tables)

[]
[('Message',)]
[('process_data_table',)]


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('process_data_table', con=engine)
X = df.iloc[:,1]
Y = df.iloc[:,4:]

In [4]:
df.iloc[:,4:].columns.nunique()

36

In [5]:
df.index += 1

In [6]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
1,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X.head(2)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

In [8]:
Y.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
Y.nunique()

related                   3
request                   2
offer                     2
aid_related               2
medical_help              2
medical_products          2
search_and_rescue         2
security                  2
military                  2
child_alone               1
water                     2
food                      2
shelter                   2
clothing                  2
money                     2
missing_people            2
refugees                  2
death                     2
other_aid                 2
infrastructure_related    2
transport                 2
buildings                 2
electricity               2
tools                     2
hospitals                 2
shops                     2
aid_centers               2
other_infrastructure      2
weather_related           2
floods                    2
storm                     2
fire                      2
earthquake                2
cold                      2
other_weather             2
direct_report       

In [10]:
Y['related'].value_counts()

1.0    19906
0.0     6122
2.0      188
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('forest', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42, verbose=1)))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size = 0.3)

In [14]:
pipeline.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   31.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   23.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   33.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   18.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   13.8s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   10.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   10.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   17.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.7s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=1, warm_start=False),
           n_jobs=1))])

In [15]:
y_pred = pipeline.predict(X_test)

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
def capture_classification_report(y_true, y_pred):
    # Redirect stdout to capture the print output
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    try:
        print(classification_report(y_true, y_pred))
    finally:
        sys.stdout = old_stdout
    return buffer.getvalue()

# Iterate through each column in the target dataframe Y
for column in Y.columns:
    report = capture_classification_report(y_test[column], y_pred[:, Y.columns.get_loc(column)])
    print(f"Category: {column}\n")
    print(report)
    print("\n" + "="*80 + "\n")

Category: related

             precision    recall  f1-score   support

        0.0       0.72      0.27      0.39      1873
        1.0       0.80      0.97      0.88      5934
        2.0       0.75      0.05      0.10        58

avg / total       0.78      0.79      0.75      7865




Category: request

             precision    recall  f1-score   support

        0.0       0.89      0.99      0.94      6533
        1.0       0.90      0.42      0.58      1332

avg / total       0.90      0.89      0.88      7865




Category: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7829
        1.0       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865




Category: aid_related

             precision    recall  f1-score   support

        0.0       0.76      0.88      0.82      4646
        1.0       0.78      0.60      0.68      3219

avg / total       0.77      0.77      0.76      7865



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
                'forest__estimator__n_estimators': [50, 200, 500],
                'forest__estimator__max_depth': [None, 10, 20, 30],
                'forest__estimator__min_samples_split': [2, 3, 4]
    
              }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1, cv=4)

In [18]:
print(cv)

GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=1, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'forest__estimator__n_estimators': [50, 200, 500], 'forest__estimator__max_depth': [None, 10, 20, 30], 'forest__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)


In [ ]:
# Fit the grid search corteiz on the low, Jesus man
cv.fit(X_train, y_train)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    9.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    7.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   10.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    6.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.0s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    8.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.0s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.3s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    7.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.0s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    7.7s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]

In [ ]:
# Best parameters
best_params = cv.best_params_
#preditcitng using best estimator 
print("Best parameters found: ", best_params)

In [ ]:
y_pred_grid = grid_search.best_estimator_.predict(X_test)

In [ ]:
histroic

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


In [ ]:
print(f"Accuracy: {accuracy}")

In [ ]:
print(f"Precision: {precision}")

In [ ]:
print(f"Recall: {recall}")

In [ ]:
print("\nClassification Report:\n", classification_report(y_test, y_pred_grid))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.